# ✅**Popular_Keywords(완료).ipynb - 문서 유사도 판단**

> 들여쓴 블록


---
### 뉴스 데이터에서 인기 있는 Keyword를 추출

* 지정한 날짜의 뉴스데이터의 제목들을 crawling  
* TF-IDF를 통해 기사 제목들 중 빈도 높은 Keyword 20개 추출

> Used Method, Skills 
* Data Clustering
* Data PreProcessing
* TF-IDF Based Document Similarity

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1
!pip3 install konlpy

  Using cached urllib3-1.26.7-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.down

In [ ]:
import os, sys
import re
import pandas as pd
import numpy as np

from google.colab import drive

from selenium import webdriver
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
base_path = "https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid="
# 실시간 검색어를 위해서는 오늘의 날짜 입력 
today = "20211207"
date_param = "&date=" + today

# 경향신문, 국민일보, 동아일보, 문화일보, 서울신문, 세계일보, 조선일보, 중앙일보, 한겨레, 한국일보
press_numbers = ["032","005","020","021","081","022","023","025","028","469",
                 "421","003","422","449","215","437","056","214","055","057",
                 "374","448","052","009","008","277","366","123","014","015",
                 "016","629","047"]
                 
# 크롤링한 제목 저장 배열 
result = []

# 언론사 당 가져올 페이지 지정
for press_number in press_numbers:
    path = base_path + press_number + date_param
    driver.get(path)
    for i in range(0, 5):
        try:
            page_button = driver.find_element_by_class_name("paging").find_elements_by_tag_name("a")
            news_list = driver.find_element_by_class_name("type06_headline").find_elements_by_tag_name("li")
            news_list += driver.find_element_by_class_name("type06").find_elements_by_tag_name("li")
            for news in news_list:
                news_title = news.find_elements_by_tag_name('a')[-1].text
                # [책과 삶], [사설], [속보] 이런 것들 삭제
                result.append(re.sub(r'\[[^)]*\]', '', news_title).strip())
            page_button[i].click()
        except:
            pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


In [ ]:
# Keyword.csv로 저장
pd.DataFrame(result).to_csv('/content/drive/MyDrive/DataMining TeamProject/Keyword.csv')

In [ ]:
# Crawling 한 "Keyword.csv" 파일에서 기사 제목을 추출 & DataFrame에 저장
df = pd.read_csv('/content/drive/MyDrive/DataMining TeamProject/Keyword.csv')
df.columns = ["num", "title"]

In [ ]:
# 불용어(STOP_WORDS)
with open('/content/drive/MyDrive/DataMining TeamProject/stopword.txt', 'r') as f:
    STOP_WORDS = f.read().replace('\n',' ').split()

# Data Preprocessing
def preprocessing(s):
  return re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

In [ ]:
# 기사 제목 전처리 후 TF-IDF 계산
df['title_cleaned'] = (df['title'].astype('U')).apply(preprocessing)

tfidf = TfidfVectorizer(stop_words=STOP_WORDS)
tfidf_matrix = tfidf.fit_transform(df['title_cleaned'])

In [ ]:
# 계산된 TF-IDF를 바탕으로 빈도가 높은 단어부터 출력
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    '빈도': tfidf_matrix.sum(axis=0).flat
})

word_count.sort_values('빈도', ascending=False).head(20)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,단어,빈도
5278,오미크론,36.185413
5819,이재명,31.855159
5643,윤석열,21.895944
8217,확진자,19.576069
5124,역대,18.069111
8186,확진,16.319797
1706,김종인,16.303361
5654,윤석열이준석,16.243501
370,3명,16.226010
4082,선대위,15.645119
